In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
path = '/content/gdrive/Othercomputers/내 노트북/Desktop/Dacon/ComputerVIsion'

import os
os.chdir(path)

!pip install timm
!pip install torchsampler

Mounted at /content/gdrive/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx

In [4]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time
from torchsampler.imbalanced import ImbalancedDatasetSampler

device = torch.device('cuda')

In [5]:
train_png = sorted(glob('open/train/*.png'))
test_png = sorted(glob('open/test/*.png'))

In [6]:
train_y = pd.read_csv("open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512))
    return img

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]


In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = self.img_paths[idx]

        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy()
            elif augmentation==2:
                img = img[:,::-1].copy()

        img = transforms.ToTensor()(img)
        if self.mode=='test':
            pass

        label = self.labels[idx]
        return img, label

    def get_labels(self):
        return self.labels


class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)

    def forward(self, x):
        x = self.model(x)
        return x


NameError: name 'Dataset' is not defined

In [ ]:
batch_size = 32
epochs = 120

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          sampler=ImbalancedDatasetSampler(train_dataset))

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)


def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score


NameError: name 'Custom_dataset' is not defined

# ImbalancedSampler Debugging

In [ ]:
# ImbalancedSampler Debugging ------------------------------------
invert_label_unique = {value:key for key, value in label_unique.items()}
df = pd.read_csv("open/train_df.csv")

for idx, sample_data in enumerate(train_loader): # batch_size 만큼 [이미지:label]튜플 불러옴
    if idx == 1: break
    # print(sample_data)""
    x = torch.tensor(sample_data[0], dtype=torch.float32, device=device)
    y = torch.tensor(sample_data[1], dtype=torch.long, device=device)

    label_names = []
    print(f"{'sampled sample_data label':<30} | {'Total Count'}")
    print("-" * 50)
    for sample in y:
        sample = sample.item() # tensor 스칼라 값을 Python의 기본 자료형(예: int, float)으로 변환
        label_name = invert_label_unique[sample]
        label_names.append(label_name)

        sampled_label_cnt = len(df[df['label'] == label_name]) # 'label' 열에서 label_name과 같은 행의 갯수
        print(f"{label_name:<30} | {sampled_label_cnt:>10}") # :<30 왼쪽 정렬,최소 30자리 공간을 차지
    print(label_names)
    print(y)
# ---------------------------------------------------------------------

NameError: name 'train_loader' is not defined

In [ ]:
model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()


best=0
train_f1_list = []
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()


    train_f1 = score_function(train_y, train_pred)
    train_f1_list.append(train_f1)
    # print(train_f1_list)
    if max(train_f1_list) > train_f1_list[-1]:
        torch.save(model.state_dict(), "./model/"+str(epoch)+".pt")

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')


[BASELINE] EfficientNet_b0을 활용한 분류 모델[public:0.67091]



---





---





---


### 라이브러리 로드

In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 장치 설정: GPU가 사용 가능한 경우 'cuda', 그렇지 않으면 'cpu'

### 데이터로드1 - png파일 load


In [3]:
train_png = sorted(glob('open/train/*.png'))
test_png = sorted(glob('open/test/*.png'))
print(len(train_png))
print(len(test_png))

4277
2154


### 데이터로드1 - train_df.csv 파일 로드 후 label 인덱스화

In [7]:
train_y = pd.read_csv("open/train_df.csv")
train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))

label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))} #zip: 두 배열 matching -> dictionary구조로(라벨:숫자)

train_labels = [label_unique[k] for k in train_labels] # train_labels를 인덱스로 변환


### 데이터로드2 - img_load() 정의 및 로드

In [8]:
def img_load(path):
    #OpenCV -> Numpy배열(height, width, 색상채널. 보통 BGR)
    img = cv2.imread(path)[:,:,::-1] #:모든요소, 모든요소, ::-1 색상채널 순서를 뒤집는다(RGB로)
    img = cv2.resize(img, (512, 512))
    return img

In [ ]:
# 이미지 로딩
# train_imgs = [img_load(m) for m in tqdm(train_png)]
# test_imgs = [img_load(n) for n in tqdm(test_png)]

train_imgs = []  #tqdm: 로딩상황 시각적확인  (List Comprehension일때 줄나눔있어서 변경) 2.37
for m in tqdm(train_png):
    train_imgs.append(img_load(m))
test_imgs = []
for n in tqdm(test_png):
    test_imgs.append(img_load(n))

  7%|▋         | 156/2154 [03:52<47:04,  1.41s/it]

### 데이터로드3 - Custom_dataset() 정의  
### Custom_dataset2() + ProcessedDataset() 정의 - train_dataset파일 생성용

In [12]:
#[JAVA] Class: 속성, 생성자, 메소드 /  접근제어자 O / 다중상속x interface통해가능 / 메소드오버로딩 O
#[Python] 생성자, 메소드 /  밑줄 / 다중상속 O / 메소드오버로딩X / class외부에서 동적 속성,메소드 추가 가능
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:
                img = img[::-1].copy() #좌우반전, 새로운배열
            elif augmentation==2:
                img = img[:,::-1].copy() #상하반전, ""

        img = transforms.ToTensor()(img) # 1.PyTorch텐서로 변환(모델입력되게) 2. 0~255 값을 0~1사이 정규화 자동수행
        if self.mode=='test':
            pass

        label = self.labels[idx]
        return img, label


In [13]:
class Custom_dataset2(Dataset):
    def __init__(self, imgs, labels, mode='train', save_path='processed_data/train_dataset.pt'):
        self.imgs = imgs
        self.labels = labels
        self.mode = mode
        self.save_path = save_path
        os.makedirs(os.path.dirname(self.save_path), exist_ok=True)

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]

        if self.mode == 'train':
            augmentation = random.randint(0, 2)
            if augmentation == 1:
                img = img[::-1].copy()  # 좌우 반전
            elif augmentation == 2:
                img = img[:, ::-1].copy()  # 상하 반전

        img_tensor = transforms.ToTensor()(img)  # PyTorch 텐서로 변환

        return img_tensor, label

    def save_processed_data(self):
        all_data = []
        for idx in range(len(self.imgs)):
            img_tensor, label = self.__getitem__(idx)
            all_data.append((img_tensor, label))
        torch.save(all_data, self.save_path)


In [14]:
class ProcessedDataset(Dataset):
    def __init__(self, data_path='processed_data/train_dataset.pt'):
        self.data = torch.load(data_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_tensor, label = self.data[idx]
        return img_tensor, label

### 모델 정의

In [15]:
class Network(nn.Module):  #pyTorch의 nn,Module을 상속받아 새로운 신경망 모듈 Network를 정의
    def __init__(self):
        super(Network, self).__init__()  #부모class(nn,Module)의 생성자 호출 -> 초기화
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88) #timm - PyTorch용 다양한 이미지 분류 모델을 제공하는 라이브러리

    def forward(self, x):
        x = self.model(x)
        return x

### 데이터로드4 : pyTorch텐서로 변환 ->

In [1]:
batch_size = 32
epochs = 25

# Train 1
# train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train') #train_imgs는 이미 numPy배열이라 불필요? (아마 데이터형식 명확히 or 복사본 필요) / 리턴: pyTorch의 Tensor
# train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size) #배치단위로 묶어서 로드.Torch.Tensor(배치크기, 채널 색상, height, width) / 마지막 배치는 다를수있음(나누어떨어지지않으면..)

# Train 2 - pt파일로 train_dataset가져올때
train_dataset = Custom_dataset2(np.array(train_imgs), np.array(train_labels), mode='train')
train_dataset.save_processed_data() # 생성 및 저장
processed_train_dataset = ProcessedDataset(data_path='processed_data/train_dataset.pt')  #불러오기
data_loader = DataLoader(processed_train_dataset, shuffle=True, batch_size=batch_size)
#Train2 끝


# Test 1
# test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
# test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Test 2 - pt파일로 train_dataset가져올때
test_dataset = Custom_dataset2(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_dataset.save_processed_data() # 생성 및 저장
processed_test_dataset = ProcessedDataset(data_path='processed_data/train_dataset.pt')  #불러오기
test_loader = DataLoader(processed_test_dataset, shuffle=False, batch_size=batch_size)
# Test 2 끝

NameError: name 'Custom_dataset2' is not defined

# **모델 학습**

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro") #'macro 평균'은 각 클래스에 대해 F1 점수를 개별적으로 계산한 후, 모든 클래스의 F1 점수의 단순 평균을
    return score

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()


best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device) # (배치크기, 채널, h, w) 형태의 텐서(이미지)
        y = torch.tensor(batch[1], dtype=torch.long, device=device) # (배치크기,)형태의 텐서type으로 변환(정답값)
        with torch.cuda.amp.autocast():  # AMP(Auto Mixed Precision) 모듈 + 모든 연산은 자동으로 적절한 정밀도로 변환됩니다. 주로 GPU 연산, 작은 물컵.
            pred = model(x)
        loss = criterion(pred, y) #손실함수(모델이 얼마나 틀렸는지 계산)

        # 손실값을 적당히 크게 해서 계산 (언더플로우 방지)
        scaler.scale(loss).backward() #역전파: 다시 예측할때 필요한 지우개
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()/len(train_loader) #학습하면서 바로바로 평균손실 계산
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist() #pred.argmax(1)는 모델이 예측한 값 중 가장 높은 확률을 가진 값을 선택 #모델이 여러 사진에 대해 예측한 결과를 모두 리스트에 모아두고, 나중에 한 번에 확인할 수 있다
        train_y += y.detach().cpu().numpy().tolist() # detach()를 사용해서 계산 그래프에서 분리, 나중에 역전파(backpropagation)할 때 영향을 받지 않는다.
        #train_y는 detach()가 y에 필요하지 않다던데..?;
        # cpu().numpy().tolist(): 텐서는 GPU연산가능, 파이썬은 불가.

    train_f1 = score_function(train_y, train_pred) #파이썬or NumPy배열 필요, return score

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

NameError: name 'Network' is not defined

# **추론**

In [ ]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device) #batch[0]: 튜플의1st. 이미지
        with torch.cuda.amp.autocast(): # PyTorch의 자동 혼합 정밀도(Automatic Mixed Precision)를 활성화
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist()) #모델의 '예측 결과' pred에서 가장 높은 값을 가지는 클래스 인덱스를 반환합니다
        # -> 그래디언트 계산을 추적하지 않도록 텐서를 분리 ->  CPU로 이동합니다. numpy 배열로 변환하기 위해 -> Python 리스트로 변환

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()} # 모든 (키, 값)을 튜플의 리스트로 반환

f_result = [label_decoder[result] for result in f_pred]

# **제출물 생성**

In [ ]:
submission = pd.read_csv("open/sample_submission.csv")

submission["label"] = f_result

submission